In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

In [ ]:
# Set the paths to the dataset directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'
val_dir = 'dataset/val'

In [ ]:
# Define the image dimensions and batch size
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# Create ImageDataGenerators for training, test, and validation sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=True
)


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle test data
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle validation data
)

In [ ]:
# Define the input shape
input_shape = train_generator.image_shape

In [ ]:
# Define the input tensor
input_img = Input(shape=input_shape)


In [ ]:
# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)


In [ ]:
# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [ ]:
# Create the Autoencoder model
autoencoder = Model(input_img, decoded)

In [ ]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the Autoencoder
history = autoencoder.fit(
    train_generator,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=val_generator
)

In [ ]:
# Evaluate the Autoencoder on the test set
test_loss = autoencoder.evaluate(test_generator)


In [ ]:
# Save the trained Autoencoder model
autoencoder.save('../autoencoder_model/autoencoder.keras')
